# Вебинар 4. Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напишите код для класса ниже
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

---

In [1]:
import numpy as np
import pandas as pd
from metrics import precision_at_k, recall_at_k

# обновляем модуль (удобно при разработке, когда вносим изменения в модуле)
import recommenders
import importlib
importlib.reload(recommenders)

from recommenders import MainRecommender


In [2]:
data = pd.read_csv('../data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                     'product_id': 'item_id'},
            inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [5]:
main_recommender = MainRecommender(data_train, prefilter_n_popular=5000, prefilter_item_features=item_features)

/Users/vladimir/DataspellProjects/GB-Recommender-Systems/Lesson_4/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 90386 to 5001


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [6]:
result['similar_items_recommendation'] = [main_recommender.get_similar_items_recommendation(user_id=x, N=5) for x in result.user_id]
result.head(5)

,user_id,actual,similar_items_recommendation
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[839818, 1007512, 9297615, 5577022, 9803545]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1044078, 1024051, 1000736, 839419, 994994]"
2,5,"[913077, 1118028, 1386668]","[12262778, 838882, 6391152, 946839, 942778]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[948650, 5569845, 1105488, 934639, 941361]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[5585510, 1044078, 12352330, 7147145, 995478]"


In [7]:
result['similar_users_recommendation'] = [main_recommender.get_similar_users_recommendation(user_id=x, N=5) for x in result.user_id]
result.head(5)

,user_id,actual,similar_items_recommendation,similar_users_recommendation
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[839818, 1007512, 9297615, 5577022, 9803545]","[856942, 1127831, 1070820, 947412, 866211]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1044078, 1024051, 1000736, 839419, 994994]","[1106523, 1029743, 8090521, 8090521, 1029743]"
2,5,"[913077, 1118028, 1386668]","[12262778, 838882, 6391152, 946839, 942778]","[874972, 1106523, 1076842, 830245, 1029743]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[948650, 5569845, 1105488, 934639, 941361]","[863447, 1113675, 959737, 1026118, 1137284]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[5585510, 1044078, 12352330, 7147145, 995478]","[1122358, 1127831, 1126899, 1070820, 6552318]"


In [8]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.053:similar_items_recommendation
0.1252:similar_users_recommendation


/Users/vladimir/opt/anaconda3/envs/DataScience/lib/python3.7/site-packages/numpy/lib/arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
